In [1]:
%reload_ext autoreload
%autoreload 2

import sys, os
import pickle

import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [2]:
# import darkhistory
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm'
sys.path.append(os.environ['DH_DIR'])
# import DM21cm
sys.path.append('..')

In [3]:
abscs = pickle.load(open('../data/abscissas/abscs_230408.p', 'rb'))

In [10]:
DATA_DIR = os.environ['DM21CM_DATA_DIR'] + '/tf/230408'

# Electrons transfer functions

## 1. Energy deposition transfer function
- `tfgv` is f values ($E\rightarrow E$). To make it a $N\rightarrow E$ transfer function, need to first multiply by the energy abscissa.
- `tfgv` is ~ (rs Ein x out).

In [11]:
os.listdir(DATA_DIR+'/elec')

['hep_tf_rxneo.npy',
 'ionhist_output',
 'lee_tf_rxneo.npy',
 '.ipynb_checkpoints',
 'lep_tf_rxneo.npy',
 'cmbloss_rxneo.npy',
 'lowerbound_rxneo.npy',
 'hed_tf_rxneo.npy']

In [7]:
abscs_in_tf = {
    'rs' : abscs['rs'],
    'Ein' : abscs['elecEk'],
    'x' : abscs['x'],
    'out' : abscs['dep_c']
}

In [8]:
data = np.load(DATA_DIR+'/elec/depgv_rexo.npy')
print(data.shape)

(20, 500, 10, 5)


In [9]:
# multiply in energy
eng = abscs['Ein']
data_NEtf = np.einsum('rexo,e->rexo', data, eng)
print(data_NEtf.shape)

(20, 500, 10, 5)


In [10]:
pickle.dump((abscs, data_NEtf), open(DATA_DIR + '/elec_dep_dlnz4.879E-2_rexo_ad.p', 'wb'))

## 2. Electron number transfer function
- `tfgv` is already in $N\rightarrow N$.
- `tfgv` is ~ (rs Ein x out).

In [12]:
abscs = {
    'rs' : abscs_nBs['rs'],
    'Ein' : abscs_nBs['elecEk'],
    'x' : abscs_nBs['x'],
    'out' : abscs_nBs['photE']
}

In [13]:
data = np.load(DATA_DIR + '/elec_phot_tfgv_rexo.npy')
print(data.shape)

(20, 500, 10, 500)


In [14]:
pickle.dump((abscs, data), open(DATA_DIR + '/elec_tf_dlnz4.879E-2_rexo_ad.p', 'wb'))

# Photons transfer functions

## 1. Energy deposition transfer function
- `tfgv` is f values ($E\rightarrow E$). To make it a $N\rightarrow E$ transfer function, need to first multiply by the energy abscissa.
- `tfgv` is ~ (nBs x rs Ein out). Transpose to make it ~ (rs Ein nBs x out).

In [12]:
os.listdir(DATA_DIR+'/phot')

['cmbloss_rxneo.npy',
 'lee_tf_rxneo.npy',
 'hep_tf_rxneo.npy',
 'ionhist_output',
 'lowerbound_rxneo.npy',
 'phot_tfgv.npy',
 'hed_tf_rxneo.npy',
 'phot_depgv.npy',
 'lep_tf_rxneo.npy']

In [13]:
abscs_in_tf = {
    'rs' : abscs['rs'],
    'Ein' : abscs['photE'],
    'nBs' : abscs['nBs'],
    'x' : abscs['x'],
    'out' : abscs['dep_c']
}

### 1.1 permute to renxo

In [16]:
data = np.load(DATA_DIR + '/phot/phot_depgv.npy') # rxneo
print(data.shape)
data_permuted = np.transpose(data, axes=[0, 3, 2, 1, 4])
print(data_permuted.shape)

(10, 10, 10, 500, 5)
(10, 500, 10, 10, 5)


In [17]:
# multiply in energy
eng = abscs_in_tf['Ein']
data_NEtf = np.einsum('renxo,e->renxo', data_permuted, eng)
print(data_NEtf.shape)

(10, 500, 10, 10, 5)


In [18]:
axes = ('rs', 'Ein', 'nBs', 'x', 'out')
pickle.dump(
    (abscs_in_tf, axes, data_NEtf),
    open(DATA_DIR + '/phot/phot_dep_renxo_aad.p', 'wb')
)

### 1.2 don't permute (nxreo)

In [ ]:
data = np.load(DATA_DIR + '/phot_depgv.npy')
print(data.shape)

In [ ]:
# multiply in energy
eng = abscs['Ein']
data_NEtf = np.einsum('nxreo,e->nxreo', data, eng)
print(data_NEtf.shape)

In [ ]:
pickle.dump((abscs, data_NEtf), open(DATA_DIR + '/phot_dep_nxreo_ad.p', 'wb'))

## 2. Photon number transfer function
- `tfgv` is already in $N\rightarrow N$.
- `tfgv` is ~ (nBs x rs Ein out). Transpose to make it ~ (rs Ein nBs x out).

In [22]:
abscs_in_tf = {
    'rs' : abscs['rs'],
    'Ein' : abscs['photE'],
    'nBs' : abscs['nBs'],
    'x' : abscs['x'],
    'out' : abscs['photE']
}

### 2.1 permute to renxo

In [23]:
data = np.load(DATA_DIR + '/phot/phot_tfgv.npy')
print(data.shape)
data_permuted = np.transpose(data, axes=[0, 3, 2, 1, 4])
print(data_permuted.shape)

(10, 10, 10, 500, 500)
(10, 500, 10, 10, 500)


In [24]:
axes = ('rs', 'Ein', 'nBs', 'x', 'out')
pickle.dump(
    (abscs_in_tf, axes, data_permuted),
    open(DATA_DIR + '/phot/phot_phot_renxo_aad.p', 'wb')
)

### 2.2 don't permute (nxreo)

In [ ]:
data = np.load(DATA_DIR + '/phot_tfgv.npy')
print(data.shape)

In [ ]:
pickle.dump((abscs, data), open(DATA_DIR + '/phot_tf_nxreo_ad.p', 'wb'))

## 3. Compound to fixed power (48)

In [17]:
from jax import jit
import jax.numpy as jnp

from tqdm import tqdm

In [18]:
@jit
def compound_48(tf, dep):
    
    dot = jnp.dot
    
    s2 = jnp.identity(tf.shape[0]) + tf
    p2 = dot(tf, tf)
    s4 = s2 + dot(p2, s2)
    p4 = dot(p2, p2)
    s8 = s4 + dot(p4, s4)
    p8 = dot(p4, p4)
    s16 = s8 + dot(p8, s8)
    p16 = dot(p8, p8)
    s32 = s16 + dot(p16, s16)
    p32 = dot(p16, p16)
    
    p48 = dot(p32, p16)
    s48 = s32 + dot(p32, s16)
    #p50 = dot(p48, p2)
    #s50 = s32 + dot(p32, s16) + dot(p48, s2)
    
    return p48, dot(s48, dep)

In [19]:
tf_abscs, tf_full = pickle.load(open(DATA_DIR + '/phot_tf_nxreo_ad.p', 'rb'))
dep_abscs, dep_full = pickle.load(open(DATA_DIR + '/phot_dep_nxreo_ad.p', 'rb'))

In [20]:
ctf_full = np.zeros_like(tf_full)
cdep_full = np.zeros_like(dep_full)

n_i, n_j, n_k, _, _ = tf_full.shape
pbar = tqdm(total=n_i*n_j*n_k)

for i in range(n_i):
    for j in range(n_j):
        for k in range(n_k):
            tf = tf_full[i,j,k]
            dep = dep_full[i,j,k]
            ctf, cdep = compound_48(tf, dep)
            ctf_full[i,j,k] = np.array(ctf)
            cdep_full[i,j,k] = np.array(cdep)
            pbar.update()

100%|█████████▉| 1999/2000 [00:27<00:00, 61.75it/s]

In [21]:
pickle.dump((tf_abscs, ctf_full), open(DATA_DIR + '/phot_ctf_dlnz48m_nxreo_ad.p', 'wb'))
pickle.dump((dep_abscs, cdep_full), open(DATA_DIR + '/phot_cdep_dlnz48m_nxreo_ad.p', 'wb'))

100%|██████████| 2000/2000 [00:43<00:00, 61.75it/s]

### 3.1 permute

In [22]:
# tf
print(ctf_full.shape, end=' -> ')
ctf_full_permuted = np.transpose(ctf_full, axes=[2, 3, 0, 1, 4])
print(ctf_full_permuted.shape)
# dep
print(cdep_full.shape, end=' -> ')
cdep_full_permuted = np.transpose(cdep_full, axes=[2, 3, 0, 1, 4])
print(cdep_full_permuted.shape)

(10, 10, 20, 500, 500) -> (20, 500, 10, 10, 500)
(10, 10, 20, 500, 5) -> (20, 500, 10, 10, 5)


In [23]:
pickle.dump((tf_abscs, ctf_full_permuted), open(DATA_DIR + '/phot_ctf_dlnz48m_renxo_ad.p', 'wb'))
pickle.dump((dep_abscs, cdep_full_permuted), open(DATA_DIR + '/phot_cdep_dlnz48m_renxo_ad.p', 'wb'))

In [24]:
! du -smh /data/submit/yitians/DM21cm/transferfunctions/nBs_test/*

3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/archive
39M	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_cdep_dlnz48m_nxreo_ad.p
39M	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_cdep_dlnz48m_renxo_ad.p
3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_ctf_dlnz48m_nxreo_ad.p
3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_ctf_dlnz48m_renxo_ad.p
39M	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_depgv.npy
39M	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_dep_nxreo_ad.p
39M	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_dep_renxo_ad.p
3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_tfgv.npy
3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_tf_nxreo_ad.p
3.8G	/data/submit/yitians/DM21cm/transferfunctions/nBs_test/phot_tf_renxo_ad.p
